In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import sys
import tensorflow as tf
from tqdm import trange

sys.path.append("/home/hongdp/Workspace/landmark")
from layers.atnconv.atnconv import AtnConv

DATA_DIR_PATH = '/mnt/shared_data/Workspace/landmark/img/9633/'
BATCH_SIZE = 1
TRAIN_STEPS = 20000
CKPT_INTERVAL = 5000
IMAGE_DIM = 256
PATCH_DIM = IMAGE_DIM // 2

In [2]:
def load_dataset():
    def preprocess_image(image):
        image = tf.image.decode_image(
            image, channels=3, dtype=tf.float32, expand_animations=False)
        image = tf.image.resize(image, [IMAGE_DIM, IMAGE_DIM])
        image.set_shape([IMAGE_DIM, IMAGE_DIM, 3])
        #image /= 255.0  # normalize to [0,1] range

        return image

    def load_and_preprocess_image(path):
        image = tf.read_file(path)
        return preprocess_image(image)

    data_root = pathlib.Path(DATA_DIR_PATH)
    all_image_paths = list(data_root.glob('*'))
    all_image_paths = [str(path) for path in all_image_paths]
    path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
    image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=8)
    image_ds = image_ds.repeat().batch(BATCH_SIZE)

    iter = image_ds.make_one_shot_iterator()
    el = iter.get_next()
    return el


def preprocess(input_img):
    mask_val = np.zeros(
        shape=[BATCH_SIZE, IMAGE_DIM, IMAGE_DIM, 1], dtype=np.float32)
    mask_val[:, ((IMAGE_DIM - PATCH_DIM)//2):((IMAGE_DIM + PATCH_DIM)//2), ((IMAGE_DIM - PATCH_DIM)//2):((IMAGE_DIM + PATCH_DIM)//2), 0] = 1
    mask = tf.constant(mask_val, dtype=tf.float32, name='mask')
    masked_img = input_img * (-mask + 1)
    return masked_img, mask

def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.get_variable("u", [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        
        """
        power iteration
        Usually iteration = 1 will be enough
        """
        
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)
        
    return w_norm

def build_discriminator(input_img, reuse=None):

    with tf.variable_scope('discriminator', reuse=reuse):
        dims = [64, 128, 256, 512, 1]
        strides = [2, 2, 2, 2, 1]
        activation = [tf.nn.leaky_relu] * 4 + [None]
        x = input_img
        for i in range(len(dims)):
            with tf.variable_scope('snconv_%d'%(i), reuse=reuse):
                w = tf.get_variable("kernel", shape=[5, 5, x.get_shape()[-1], dims[i]])
                b = tf.get_variable("bias", [dims[i]], initializer=tf.constant_initializer(0.0))
                x = tf.nn.conv2d(input=x, filter=spectral_norm(w), strides=[1, strides[i], strides[i], 1], padding='SAME') + b
                if activation[i]:
                    x = activation[i](x)

        logits = tf.layers.dense(tf.layers.flatten(x), 1)
    return logits


def build_generator(input_img, mask, reuse=None):

    with tf.variable_scope('generator', reuse=reuse):
        cnum = 32
        x = tf.concat([input_img, mask], axis=3)
        x = tf.layers.conv2d(x, filters=cnum//2,
                             kernel_size=3, strides=1, padding='SAME', activation=tf.nn.leaky_relu)

        # encode
        enc_feats = []
        dims = [cnum * i for i in [1, 2, 4, 8, 8, 8]]
        activations = [tf.nn.leaky_relu] * 6 + [tf.nn.relu]
        for i in range(len(dims)):
            enc_feats.append(x)
            x = tf.layers.conv2d(
                x, filters=dims[i], kernel_size=3, strides=2, padding='SAME', activation=activations[i])
        latent_feat = x

        # attention transfer networks
        attn_feats = []
        x = latent_feat
        for i in range(len(dims)):
            x = AtnConv(enc_feats[-(i+1)], x,
                        tf.expand_dims(mask[0, :, :, :], 0))
            attn_feats.append(x)

        # decode
        x = latent_feat
        dims = [cnum * i for i in [1./2, 1, 2, 4, 8, 8]]
        outputs = [None] * len(dims)
        for i in range(len(dims)):
            attn_feats_shape = tf.shape(attn_feats[i])
            x = tf.layers.conv2d_transpose(x, filters=int(
                dims[-(i+1)]), kernel_size=3, strides=2, padding='SAME', activation=tf.nn.relu)
            x = tf.concat([x, attn_feats[i]], axis=3)
            outputs[-(i+1)] = tf.clip_by_value(tf.layers.conv2d(x, filters=3,
                                                                kernel_size=1, strides=1, padding='SAME'), 0, 1)
        final_output = input_img + outputs[0] * mask
    return outputs, final_output


def build_network(input_img):
    # input_img = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMAGE_DIM, IMAGE_DIM, 3])
    processed_input, mask = preprocess(input_img)

    layer_outputs, final_output = build_generator(processed_input, mask)
    real_logits = build_discriminator(input_img)
    fake_logits = build_discriminator(final_output, reuse=True)
    d_loss = tf.maximum(.0, 1 - real_logits) + tf.maximum(.0, 1 + fake_logits)

    l1_losses = []
    for output in layer_outputs:
        output_shape = tf.shape(output)
        l1_losses.append(tf.losses.absolute_difference(
            tf.image.resize_images(
                input_img,
                size=[output_shape[1], output_shape[2]]),
            output))
    g_loss = tf.add_n(l1_losses) - fake_logits

    
    logdir = "./logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    for l1_layer in range(len(l1_losses)):
        tf.summary.scalar('l1_loss_%d'%l1_layer,  tf.reduce_mean(l1_losses[l1_layer]))
        tf.summary.image('output_%d'%l1_layer,layer_outputs[l1_layer])
    tf.summary.scalar('fake_logits',  tf.reduce_mean(fake_logits))
    tf.summary.scalar('real_logits',  tf.reduce_mean(real_logits))
    tf.summary.scalar('g_loss',  tf.reduce_mean(g_loss))
    tf.summary.scalar('d_loss',  tf.reduce_mean(d_loss))
    tf.summary.image('processed_input',processed_input)
    tf.summary.image('final_output',final_output)
    tf.summary.image('input',input_img)

    merged = tf.summary.merge_all()

    return g_loss, d_loss, merged


In [3]:
dataset_batch = load_dataset()
g_loss, d_loss, summary = build_network(dataset_batch)
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
global_step_tensor = tf.train.get_or_create_global_step()
train_g_op = optimizer.minimize(g_loss, var_list=tf.get_collection(
    tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator'), global_step=global_step_tensor)
train_d_op = optimizer.minimize(d_loss, var_list=tf.get_collection(
    tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator'))



W0819 17:39:08.452049 140616070059840 deprecation.py:323] From <ipython-input-2-83dd493f6984>:22: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W0819 17:39:08.538870 140616070059840 deprecation.py:323] From <ipython-input-2-83dd493f6984>:92: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0819 17:39:08.540287 140616070059840 deprecation.py:506] From /home/hongdp/Workspace/landmark/.env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dt

In [ ]:
RESTORE_PATH = "./model/model_test_2/model.ckpt-245001"
RUN='model_test_2'

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
saver = tf.train.Saver()

logdir = "./logs/" + RUN
train_writer = tf.summary.FileWriter(logdir,sess.graph)

if RESTORE_PATH:
    saver.restore(sess, RESTORE_PATH)
else:
    sess.run(tf.global_variables_initializer())
for i in trange(230000):
    _, _, summary_val = sess.run([train_g_op, train_d_op, summary])
    if not i%100:
        train_writer.add_summary(summary_val, tf.compat.v1.train.global_step(sess, global_step_tensor))
    if not i%CKPT_INTERVAL:
        save_path = saver.save(sess, "./model/%s/model.ckpt"%(RUN), global_step=global_step_tensor)

save_path = saver.save(sess, "./model/%s/model.ckpt"%(RUN), global_step=global_step_tensor)
        



  0%|          | 0/230000 [00:00<?, ?it/s]

  0%|          | 1/230000 [00:03<236:58:25,  3.71s/it]

  0%|          | 2/230000 [00:03<171:07:09,  2.68s/it]

  0%|          | 3/230000 [00:04<124:53:11,  1.95s/it]

  0%|          | 4/230000 [00:04<92:45:05,  1.45s/it] 

  0%|          | 5/230000 [00:04<70:16:59,  1.10s/it]

  0%|          | 6/230000 [00:05<54:14:58,  1.18it/s]

  0%|          | 7/230000 [00:05<42:58:54,  1.49it/s]

  0%|          | 8/230000 [00:05<35:07:52,  1.82it/s]

  0%|          | 9/230000 [00:05<29:47:59,  2.14it/s]

  0%|          | 10/230000 [00:06<25:54:36,  2.47it/s]

  0%|          | 11/230000 [00:06<23:22:23,  2.73it/s]

  0%|          | 12/230000 [00:06<21:37:36,  2.95it/s]

  0%|          | 13/230000 [00:06<20:13:57,  3.16it/s]

  0%|          | 14/230000 [00:07<19:22:47,  3.30it/s]

  0%|          | 15/230000 [00:07<19:00:06,  3.36it/s]

  0%|          | 16/230000 [00:07<18:34:06,  3.44it/s]

  0%|          | 17/230000 [00:08<18:04:19,  3.53it/s]

  0%|  

  0%|          | 131/230000 [00:39<16:55:17,  3.77it/s]

  0%|          | 132/230000 [00:39<16:52:27,  3.78it/s]

  0%|          | 133/230000 [00:39<16:53:29,  3.78it/s]

  0%|          | 134/230000 [00:39<16:50:48,  3.79it/s]

  0%|          | 135/230000 [00:40<16:49:28,  3.80it/s]

  0%|          | 136/230000 [00:40<16:51:34,  3.79it/s]

  0%|          | 137/230000 [00:40<16:51:58,  3.79it/s]

  0%|          | 138/230000 [00:41<16:54:21,  3.78it/s]

  0%|          | 139/230000 [00:41<16:52:28,  3.78it/s]

  0%|          | 140/230000 [00:41<16:59:37,  3.76it/s]

  0%|          | 141/230000 [00:41<16:58:41,  3.76it/s]

  0%|          | 142/230000 [00:42<17:08:25,  3.73it/s]

  0%|          | 143/230000 [00:42<17:04:49,  3.74it/s]

  0%|          | 144/230000 [00:42<17:05:19,  3.74it/s]

  0%|          | 145/230000 [00:42<17:03:09,  3.74it/s]

  0%|          | 146/230000 [00:43<17:11:04,  3.72it/s]

  0%|          | 147/230000 [00:43<17:15:40,  3.70it/s]

  0%|          | 148/230000 [00

  0%|          | 261/230000 [01:14<17:00:17,  3.75it/s]

  0%|          | 262/230000 [01:14<16:57:43,  3.76it/s]

  0%|          | 263/230000 [01:14<16:53:52,  3.78it/s]

  0%|          | 264/230000 [01:14<17:06:00,  3.73it/s]

  0%|          | 265/230000 [01:15<17:01:42,  3.75it/s]

  0%|          | 266/230000 [01:15<17:07:44,  3.73it/s]

  0%|          | 267/230000 [01:15<17:14:25,  3.70it/s]

  0%|          | 268/230000 [01:15<17:06:24,  3.73it/s]

  0%|          | 269/230000 [01:16<17:02:50,  3.74it/s]

  0%|          | 270/230000 [01:16<17:04:33,  3.74it/s]

  0%|          | 271/230000 [01:16<17:08:55,  3.72it/s]

  0%|          | 272/230000 [01:17<17:14:41,  3.70it/s]

  0%|          | 273/230000 [01:17<17:05:55,  3.73it/s]

  0%|          | 274/230000 [01:17<16:58:18,  3.76it/s]

  0%|          | 275/230000 [01:17<16:56:47,  3.77it/s]

  0%|          | 276/230000 [01:18<16:55:23,  3.77it/s]

  0%|          | 277/230000 [01:18<17:02:10,  3.75it/s]

  0%|          | 278/230000 [01

  0%|          | 391/230000 [01:49<17:02:17,  3.74it/s]

  0%|          | 392/230000 [01:49<17:12:45,  3.71it/s]

  0%|          | 393/230000 [01:49<17:17:28,  3.69it/s]

  0%|          | 394/230000 [01:49<17:20:36,  3.68it/s]

  0%|          | 395/230000 [01:50<17:09:25,  3.72it/s]

  0%|          | 396/230000 [01:50<17:02:45,  3.74it/s]

  0%|          | 397/230000 [01:50<17:09:42,  3.72it/s]

  0%|          | 398/230000 [01:50<17:14:54,  3.70it/s]

  0%|          | 399/230000 [01:51<17:18:23,  3.69it/s]

  0%|          | 400/230000 [01:51<17:11:39,  3.71it/s]

  0%|          | 401/230000 [01:51<17:10:48,  3.71it/s]

  0%|          | 402/230000 [01:51<17:06:30,  3.73it/s]

  0%|          | 403/230000 [01:52<17:08:36,  3.72it/s]

  0%|          | 404/230000 [01:52<17:05:59,  3.73it/s]

  0%|          | 405/230000 [01:52<17:08:06,  3.72it/s]

  0%|          | 406/230000 [01:53<17:15:39,  3.69it/s]

  0%|          | 407/230000 [01:53<17:17:10,  3.69it/s]

  0%|          | 408/230000 [01

  0%|          | 521/230000 [02:24<17:06:35,  3.73it/s]

  0%|          | 522/230000 [02:24<17:07:16,  3.72it/s]

  0%|          | 523/230000 [02:24<17:13:07,  3.70it/s]

  0%|          | 524/230000 [02:24<17:09:47,  3.71it/s]

  0%|          | 525/230000 [02:25<17:07:52,  3.72it/s]

  0%|          | 526/230000 [02:25<17:08:43,  3.72it/s]

  0%|          | 527/230000 [02:25<17:13:48,  3.70it/s]

  0%|          | 528/230000 [02:25<17:16:23,  3.69it/s]

  0%|          | 529/230000 [02:26<17:09:44,  3.71it/s]

  0%|          | 530/230000 [02:26<17:05:45,  3.73it/s]

  0%|          | 531/230000 [02:26<17:03:52,  3.74it/s]

  0%|          | 532/230000 [02:27<17:12:37,  3.70it/s]

  0%|          | 533/230000 [02:27<17:14:51,  3.70it/s]

  0%|          | 534/230000 [02:27<17:10:21,  3.71it/s]

  0%|          | 535/230000 [02:27<17:03:51,  3.74it/s]

  0%|          | 536/230000 [02:28<17:00:00,  3.75it/s]

  0%|          | 537/230000 [02:28<17:00:21,  3.75it/s]

  0%|          | 538/230000 [02

...

  4%|▍         | 9872/230000 [44:35<16:34:30,  3.69it/s]

  4%|▍         | 9873/230000 [44:35<16:39:33,  3.67it/s]

  4%|▍         | 9874/230000 [44:36<16:40:25,  3.67it/s]

  4%|▍         | 9875/230000 [44:36<16:30:54,  3.70it/s]

  4%|▍         | 9876/230000 [44:36<16:38:51,  3.67it/s]

  4%|▍         | 9877/230000 [44:36<16:38:30,  3.67it/s]

  4%|▍         | 9878/230000 [44:37<16:39:37,  3.67it/s]

  4%|▍         | 9879/230000 [44:37<16:47:01,  3.64it/s]

  4%|▍         | 9880/230000 [44:37<17:13:16,  3.55it/s]

  4%|▍         | 9881/230000 [44:38<17:37:20,  3.47it/s]

  4%|▍         | 9882/230000 [44:38<17:53:25,  3.42it/s]

  4%|▍         | 9883/230000 [44:38<17:28:03,  3.50it/s]

  4%|▍         | 9884/230000 [44:38<17:09:43,  3.56it/s]

  4%|▍         | 9885/230000 [44:39<17:02:11,  3.59it/s]

  4%|▍         | 9886/230000 [44:39<17:25:19,  3.51it/s]

  4%|▍         | 9887/230000 [44:39<17:27:04,  3.50it/s]

  4%|▍         | 9888/230000 [44:40<17:18:37,  3.53it/s]

  4%|▍        

  4%|▍         | 10000/230000 [45:10<16:25:02,  3.72it/s]

  4%|▍         | 10001/230000 [45:11<26:18:24,  2.32it/s]

  4%|▍         | 10002/230000 [45:11<23:29:51,  2.60it/s]

  4%|▍         | 10003/230000 [45:12<21:48:09,  2.80it/s]

  4%|▍         | 10004/230000 [45:12<20:18:43,  3.01it/s]

  4%|▍         | 10005/230000 [45:12<19:20:13,  3.16it/s]

  4%|▍         | 10006/230000 [45:13<18:33:25,  3.29it/s]

  4%|▍         | 10007/230000 [45:13<17:53:40,  3.41it/s]

  4%|▍         | 10008/230000 [45:13<17:31:51,  3.49it/s]

  4%|▍         | 10009/230000 [45:13<17:20:05,  3.53it/s]

  4%|▍         | 10010/230000 [45:14<17:11:26,  3.55it/s]

  4%|▍         | 10011/230000 [45:14<17:06:11,  3.57it/s]

  4%|▍         | 10012/230000 [45:14<16:59:00,  3.60it/s]

  4%|▍         | 10013/230000 [45:14<16:53:46,  3.62it/s]

  4%|▍         | 10014/230000 [45:15<16:50:47,  3.63it/s]

  4%|▍         | 10015/230000 [45:15<16:43:58,  3.65it/s]

  4%|▍         | 10016/230000 [45:15<16:34:07,  3.69it/s

  4%|▍         | 10126/230000 [45:46<16:39:00,  3.67it/s]

  4%|▍         | 10127/230000 [45:46<16:44:18,  3.65it/s]

  4%|▍         | 10128/230000 [45:46<16:36:24,  3.68it/s]

  4%|▍         | 10129/230000 [45:46<16:34:38,  3.68it/s]

  4%|▍         | 10130/230000 [45:47<16:40:33,  3.66it/s]

  4%|▍         | 10131/230000 [45:47<16:43:44,  3.65it/s]

  4%|▍         | 10132/230000 [45:47<16:37:51,  3.67it/s]

  4%|▍         | 10133/230000 [45:48<16:31:28,  3.70it/s]

  4%|▍         | 10134/230000 [45:48<16:27:36,  3.71it/s]

  4%|▍         | 10135/230000 [45:48<16:36:24,  3.68it/s]

  4%|▍         | 10136/230000 [45:48<16:33:54,  3.69it/s]

  4%|▍         | 10137/230000 [45:49<16:34:41,  3.68it/s]

  4%|▍         | 10138/230000 [45:49<17:24:13,  3.51it/s]

  4%|▍         | 10139/230000 [45:49<17:13:04,  3.55it/s]

  4%|▍         | 10140/230000 [45:50<16:59:02,  3.60it/s]

  4%|▍         | 10141/230000 [45:50<16:48:40,  3.63it/s]

  4%|▍         | 10142/230000 [45:50<16:42:35,  3.65it/s

  4%|▍         | 10252/230000 [46:21<16:59:22,  3.59it/s]

  4%|▍         | 10253/230000 [46:21<17:13:44,  3.54it/s]

  4%|▍         | 10254/230000 [46:21<17:10:37,  3.55it/s]

  4%|▍         | 10255/230000 [46:22<17:04:40,  3.57it/s]

  4%|▍         | 10256/230000 [46:22<17:04:18,  3.58it/s]

  4%|▍         | 10257/230000 [46:22<16:57:28,  3.60it/s]

  4%|▍         | 10258/230000 [46:23<16:55:42,  3.61it/s]

  4%|▍         | 10259/230000 [46:23<16:53:34,  3.61it/s]

  4%|▍         | 10260/230000 [46:23<16:47:11,  3.64it/s]

  4%|▍         | 10261/230000 [46:23<16:47:23,  3.64it/s]

  4%|▍         | 10262/230000 [46:24<16:39:18,  3.66it/s]

  4%|▍         | 10263/230000 [46:24<16:39:04,  3.67it/s]

  4%|▍         | 10264/230000 [46:24<16:35:39,  3.68it/s]

  4%|▍         | 10265/230000 [46:25<16:37:22,  3.67it/s]

  4%|▍         | 10266/230000 [46:25<16:41:22,  3.66it/s]

  4%|▍         | 10267/230000 [46:25<16:41:11,  3.66it/s]

  4%|▍         | 10268/230000 [46:25<16:41:49,  3.66it/s

  5%|▍         | 10378/230000 [46:55<16:55:54,  3.60it/s]

  5%|▍         | 10379/230000 [46:56<16:58:29,  3.59it/s]

  5%|▍         | 10380/230000 [46:56<16:49:02,  3.63it/s]

  5%|▍         | 10381/230000 [46:56<16:39:22,  3.66it/s]

  5%|▍         | 10382/230000 [46:56<17:13:20,  3.54it/s]

  5%|▍         | 10383/230000 [46:57<16:57:36,  3.60it/s]

  5%|▍         | 10384/230000 [46:57<16:55:36,  3.60it/s]

  5%|▍         | 10385/230000 [46:57<16:55:59,  3.60it/s]

  5%|▍         | 10386/230000 [46:58<17:11:17,  3.55it/s]

  5%|▍         | 10387/230000 [46:58<16:56:23,  3.60it/s]

  5%|▍         | 10388/230000 [46:58<16:52:00,  3.62it/s]

  5%|▍         | 10389/230000 [46:58<16:43:47,  3.65it/s]

  5%|▍         | 10390/230000 [46:59<16:34:50,  3.68it/s]

  5%|▍         | 10391/230000 [46:59<17:03:25,  3.58it/s]

  5%|▍         | 10392/230000 [46:59<17:02:03,  3.58it/s]

  5%|▍         | 10393/230000 [47:00<16:51:26,  3.62it/s]

  5%|▍         | 10394/230000 [47:00<16:40:40,  3.66it/s